In [ ]:
## 他の商品でもやってみよう
import pandas as pd
import numpy as np
from quantopian.research.experimental import continuous_future, history
import matplotlib.pyplot as plt 
import matplotlib.cm as cm


start = "2015-1-1"
end = "2017-12-12"

## 取得する先物の各限月のシンボルを取得
def get_future_contract_info(root_symbol_name, years):
    month_code = list("FGHJKMNQUVXZ")
    contracts = [root_symbol_name + code + str(year) for year in years for code in month_code]
    l = list()
    for c in contracts:
        print "Fetching ", c
        try:
            sym = symbols(c)
            l.append(sym.to_dict())
            
        except: 
            print "NoSuchSymbols", c
    df_future_contracts = pd.DataFrame(l)
    return df_future_contracts

## 各限月のプライスを取得
def get_daily_future_price(df_future_contracts, start, end):
    syms = df_future_contracts["symbol"].tolist()
    df_future_prices = history(syms, fields='price', frequency='daily', 
                     start=start, end=end)
    df_future_prices.columns = map(lambda x: x.symbol, df_future_prices.columns)
    return df_future_prices 

## 残存期間を取得して追加
def insert_remain_dates(df_future_prices, df_future_contracts):
    l = list()
    for date in df_future_prices.index:
        for expiry in df_future_contracts["expiration_date"]:
            if date < expiry:
                l.append(expiry)
                break 
                
    df_future_prices["Expiration"] = l
    df_future_prices["hoge"] = df_future_prices.index
    df_future_prices["Remain date"] = df_future_prices["Expiration"] - df_future_prices["hoge"]
    df_future_prices.drop("hoge", axis=1, inplace=True)
    return df_future_prices


def get_future_contract_information(root_symbol_name, start, end, years):
    df_future_contract_information = get_future_contract_info(root_symbol_name, years)
    df_future_contract_price = get_daily_future_price(df_future_contract_information, start, end)
    df_future_contract_price = insert_remain_dates(df_future_contract_price, df_future_contract_information)
    return df_future_contract_price
    
    
def get_continuous_future_price(root_symbol_name, start, end):
    f1 = continuous_future(root_symbol_name)
    f2 = continuous_future(root_symbol_name, offset=1)
    f3 = continuous_future(root_symbol_name, offset=2)
    df_continuous_future_price = history([f1,f2,f3], fields='price', frequency='daily', start=start,  end=end)
    df_continuous_future_price.columns = map(lambda x:"%s_%s" % (x.root_symbol, x.offset + 1), 
                                     df_continuous_future_price.columns)
    return df_continuous_future_price

def merge_and_insert_future_data(df_future_contract_price, df_continuous_future_price):
    root = df_continuous_future_price.columns[0].split("_")[0]
    df_continuous_future_price2 = pd.concat([df_continuous_future_price, df_future_contract_price[["Expiration", "Remain date"]]], axis=1)
    
    df_continuous_future_price2["diff1"] = (df_continuous_future_price2[root+"_2"] / df_continuous_future_price2[root+"_1"]).apply(np.log)
    df_continuous_future_price2["diff2"] = (df_continuous_future_price2[root+"_3"] / df_continuous_future_price2[root+"_2"]).apply(np.log)
    df_continuous_future_price2["diff3"] = (df_continuous_future_price2[root+"_3"] / df_continuous_future_price2[root+"_1"]).apply(np.log)
    
    df_continuous_future_price2["diff1 annualize"]  = df_continuous_future_price2["diff1"] * df_continuous_future_price2["Remain date"].dt.days / 250
    df_continuous_future_price2["diff2 annualize"]  = df_continuous_future_price2["diff2"] * df_continuous_future_price2["Remain date"].dt.days / 250
    df_continuous_future_price2["diff3 annualize"]  = df_continuous_future_price2["diff3"] * df_continuous_future_price2["Remain date"].dt.days / 250
    
    df_continuous_future_price2[root+"_1_pct_change"] = df_continuous_future_price2[root+"_1"].pct_change()
    df_continuous_future_price2[root+"_2_pct_change"] = df_continuous_future_price2[root+"_2"].pct_change()
    df_continuous_future_price2[root+"_3_pct_change"] = df_continuous_future_price2[root+"_3"].pct_change()

    return df_continuous_future_price2

    

In [ ]:
root_symbol_name, start, end, years = 'CL', "2015-1-1", "2017-12-12", [15,16,17,18]
CL_df_future_contract_price = get_future_contract_information(root_symbol_name, start, end, years)
CL_df_continuous_future_price = get_continuous_future_price(root_symbol_name, start, end)
CL_df_continuous_future_price2 = merge_and_insert_future_data(CL_df_future_contract_price, CL_df_continuous_future_price)

In [ ]:
df = get_future_contract_info("CN", [15,16,17])


In [ ]:
l = list() 
for i, row in df[["expiration_date", "start_date", "symbol"]].iterrows():
    start = row["start_date"]
    end = row["expiration_date"]
    syms = row["symbol"]
    df_future_prices = history(syms, fields='price', frequency='daily', start=start, end=end)
    l.append(df_future_prices)
    


In [ ]:
for df in l:
    plt.plot(df[-30:].apply(np.log).pct_change().cumsum().tolist())
    

In [ ]:
fig, ax = plt.subplots()
ax.plot(CL_df_continuous_future_price2["diff2 annualize"] - CL_df_continuous_future_price2["diff1 annualize"],)
ax2 = ax.twinx()
ax2.plot(uso.price,c = 'red')


In [ ]:
uso = get_pricing("USO", start_date=start, end_date=end)


In [ ]:
fig, ax = plt.subplots()
ax.scatter(CL_df_continuous_future_price2["diff2 annualize"] - CL_df_continuous_future_price2["diff1 annualize"]
           , CL_df_continuous_future_price2["CL_1"].pct_change()
           ,c = CL_df_continuous_future_price2["Remain date"].dt.days
           ,cmap = cm.Reds
          )
#ax2 = ax.twinx()
#ax2.plot(uso.price,c = 'red')

